In [1]:
# others
import pandas as pd
import numpy as np
import argparse
import datetime
from copy import deepcopy # Add Deepcopy for args
import pickle 
import seaborn as sns 
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error

# pytorch
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim


print(torch.__version__)
%matplotlib inline
%pylab inline
pylab.rcParams['figure.figsize'] = (15, 9)


# read file
df = pd.read_hdf('./data/merged_data.h5')

1.8.1
Populating the interactive namespace from numpy and matplotlib


In [2]:
''' greedy feature handleing'''
# test_df = train_x_df[train_x_df['volume'] != 0]
# test_df['rest_asset'] = test_df['volume'] - test_df['tb_base_av']
# test_df['greedy'] = test_df['tb_base_av'] / test_df['volume']

# test_df2 = test_df[['time', 'coin_index', 'open', 'high', 'low', 'close', 'volume', 'trades', 'tb_base_av','rest_asset', 'greedy']]
# test_df2[['coin_index','trades', 'volume', 'tb_base_av','rest_asset', 'greedy']].head()
# test_df2[test_df2['greedy'] == 1][['coin_index','trades', 'volume', 'tb_base_av','rest_asset', 'greedy']].head()

' greedy feature handleing'

In [4]:
train_df = df[df.sample_id < 100]
# val_df = df[(df.sample_id >= 1000) & (df.sample_id < 1108)]
# test_df = df[df.sample_id >= 1108]

# # 개수 체크
# print(
#     f'''
#     train set is {len(train_df) / 1500}
#     val set is {len(val_df) / 1500}
#     test set is {len(test_df)/1500}
#     '''
# )

In [5]:
# ====== Generating Dataset ====== #

def data_gernerate(dataframe, x_frames, y_frames, print_mode = False):

    ''' 설명 생략 '''

    # grouping
    grouped_df = dataframe.groupby('sample_id')
    
    # export unique sample ids
    unique_sample_id_list = grouped_df.sample_id.unique()

    # create new lists
    X, y = list(), list()

    ''' 샘플 하나 선택 loop '''
    for sample_id in unique_sample_id_list:
        
        # get one sample_id in sample list
        temp_sample_id = sample_id.item()

        # get one group by temp_sample_id
        temp_df = grouped_df.get_group(temp_sample_id)

        # 한 샘플당 몇 개의 arrset가 나오는 지 확인
        count = 0
        split_length = len(temp_df) - (x_frames + y_frames) + 1
        
        ''' 한 샘플 내 데이터 split loop '''
        for time_idx in range(split_length):
            
            # index 변경
            time_idx += x_frames
            
            # temp_data select
            temp_arr = temp_df.iloc[time_idx - x_frames : time_idx + y_frames, 3:].values

            # get values
            temp_x = temp_arr[:x_frames, :]
            temp_y = temp_arr[x_frames:, :]

#             # 2d to 3d -> (255, 12) to (1, 255, 12) / (120, 12) to (1, 120, 12)
#             temp_3d_x = np.expand_dims(temp_2d_x, axis = 0)
#             temp_3d_y = np.expand_dims(temp_2d_y, axis = 0)
            
            # appending
            X.append(temp_x)
            y.append(temp_y)
            
            # counter printing
            count += 1
            if (count == split_length) & (print_mode == True):
                print(f'현재 sample id : {temp_sample_id}')
                print(f'{temp_sample_id}번째 sample의 생성 array수 : {count}')
            
    return np.array(X), np.array(y)

In [6]:
train_x_arr, train_y_arr = data_gernerate(dataframe = train_df, x_frames = 255, y_frames = 120)

In [17]:
print(
    f'''
    {train_x_arr.shape}
    {train_y_arr[:,:,0].shape}
    '''
)



    (112600, 255, 9)
    (112600, 120)
    


In [8]:
import encoder_decoder

# ====== initialization
parser = argparse.ArgumentParser()
args = parser.parse_args("")
args.device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("device is",args.device)

# ====== Model Capacity options ===== #
args.input_dim = 9
args.hidden_dim = 50
args.output_dim = 1
args.num_layers = 1
args.batch_size = 64
args.dropout = 0.2
args.target_len = 120
args.use_bn = True

# ====== Dataset Generating options ====== #
args.x_frames = 255
args.y_frames = 120

# ====== Model training options ===== #
args.num_epoch = 30
args.learning_rate = 0.001
args.L2_rate = 0.0001
args.training_prediction = 'mixed_teacher_forcing'
args.teacher_forcing_ratio = 0.6

device is cpu


In [11]:
# Encoder
enc = encoder_decoder.Encoder(args.input_dim, args.hidden_dim, args.num_layers)
# Decoder
dec = encoder_decoder.Decoder(args.output_dim, args.hidden_dim, args.num_layers, args.dropout, args.use_bn)

# Seq2Seq model
model = encoder_decoder.Seq2Seq(enc, dec, args.device, args.target_len, args.training_prediction, args.teacher_forcing_ratio)
model.to(args.device)


loss_fn = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=args.learning_rate, weight_decay=args.L2_rate)
    

In [15]:
train_x_arr.shape

(112600, 255, 9)

In [14]:
for x, y in train_x_arr, train_y_arr:
    print(x.shape)
    break

ValueError: too many values to unpack (expected 2)

In [12]:

for i in range(args.num_epoch):
    model.train()
    model.zero_grad()
    optimizer.zero_grad()
    
    model.hidden = model.init_hidden()
    
    for x in train_x_arr:
        x = torch.Tensor(x).float()
        y_true = torch.Tensor(y_true).float()

        y_pred, hidden = model(x)
        model.hidden = hidden

    loss = loss_fn(y_pred.view(-1), y_true.view(-1))
    loss.backward()
    optimizer.step()
    print(loss.item())
    
    
        for i, (X, y) in enumerate(trainloader):
        
        X = X.transpose(0, 1).float().to(args.device)
        y_true = y[:,:,0].transpose(0, 1).float().to(args.device)

        # zero the gradient
        optimizer.zero_grad()
        
        # en-decoder outputs tensor 
        y_pred = model(X, y_true)

        # compute the loss 
        loss = loss_fn(y_true, y_pred)
        
        #backpropagation
        loss.backward()
        optimizer.step()
        
        # get the batch loss
        train_loss += loss.item()

AttributeError: 'Seq2Seq' object has no attribute 'init_hidden'

In [ ]:
# numpy arrary conversion

def df2d_to_array3d(df_2d):
    # 입력 받은 2차원 데이터 프레임을 3차원 numpy array로 변경하는 함수
    feature_size = df_2d.iloc[:,2:].shape[1]
    time_size = len(df_2d.time.value_counts())
    sample_size = len(df_2d.sample_id.value_counts())

    # 2위 팀에서 임의로 넣어둠
    # sample_index = df_2d.sample_id.value_counts().index
    array_3d = df_2d.iloc[:,2:].values.reshape([sample_size, time_size, feature_size])
    return array_3d


In [ ]:
print(
    f'''
    {df.high.max()}
    {df.low.max()}
    {df.open.max()}
    {df.close.max()}
    
    
    {df.high.min()}
    {df.low.min()}
    {df.open.min()}
    {df.close.min()}
    
    '''
    
    ''' high - low = 변동폭 \n'''
    ''' 음봉양봉 구분 추가 가능'''
)
